## Import libraries

In [1]:
# Importing the library
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
from IPython.display import display # Manage multiple output per cell
import datetime
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss
from sklearn.cross_validation import KFold
from sklearn import preprocessing

/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Configuration

In [2]:
odd_H = 'INFO_BbAvH'
odd_A = 'INFO_BbAvA'
odd_D = 'INFO_BbAvD'
target = 'INFO_FTR'
start_date = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")
season_list = [2014, 2015, 2016]
league = 'D1'
classes = ['A', 'D', 'H']

In [3]:
best_params = {
    'C': 8.291,
    'penalty': 'l2',
    'class_weight': None,
    'solver': 'sag',
    'max_iter': 270,
    'multi_class': 'multinomial'
}

In [4]:
best_features_MLP = ['A_MEANS_FIVE_AC', 'A_MEANS_FIVE_AS', 'A_MEANS_FIVE_AST','A_MEANS_FIVE_FTAG', 'A_MEANS_FIVE_FTHG', 'A_MEANS_FIVE_FTR_H','A_MEANS_FIVE_HC', 'A_MEANS_FIVE_HS', 'A_MEANS_FIVE_HST','A_MEANS_FIVE_HTR_A', 'H_MEANS_FIVE_AC', 'H_MEANS_FIVE_AS','H_MEANS_FIVE_AST', 'H_MEANS_FIVE_AY', 'H_MEANS_FIVE_FTAG','H_MEANS_FIVE_FTHG', 'H_MEANS_FIVE_FTR_A', 'H_MEANS_FIVE_FTR_H','H_MEANS_FIVE_HC', 'H_MEANS_FIVE_HS', 'H_MEANS_FIVE_HST','H_MEANS_FIVE_HTR_H', 'A_MEANS_THREE_AC', 'A_MEANS_THREE_AS','A_MEANS_THREE_FTHG', 'A_MEANS_THREE_HS', 'H_MEANS_THREE_AS','A_STD_FIVE_HF', 'H_STD_FIVE_HC', 'H_STD_FIVE_HST']
all_features = ["A_MEANS_FIVE_AC","A_MEANS_FIVE_AF","A_MEANS_FIVE_AR","A_MEANS_FIVE_AS","A_MEANS_FIVE_AST","A_MEANS_FIVE_AY","A_MEANS_FIVE_FTAG","A_MEANS_FIVE_FTHG","A_MEANS_FIVE_FTR_A","A_MEANS_FIVE_FTR_D","A_MEANS_FIVE_FTR_H","A_MEANS_FIVE_HC","A_MEANS_FIVE_HF","A_MEANS_FIVE_HR","A_MEANS_FIVE_HS","A_MEANS_FIVE_HST","A_MEANS_FIVE_HTAG","A_MEANS_FIVE_HTHG","A_MEANS_FIVE_HTR_A","A_MEANS_FIVE_HTR_D","A_MEANS_FIVE_HTR_H","A_MEANS_FIVE_HY","H_MEANS_FIVE_AC","H_MEANS_FIVE_AF","H_MEANS_FIVE_AR","H_MEANS_FIVE_AS","H_MEANS_FIVE_AST","H_MEANS_FIVE_AY","H_MEANS_FIVE_FTAG","H_MEANS_FIVE_FTHG","H_MEANS_FIVE_FTR_A","H_MEANS_FIVE_FTR_D","H_MEANS_FIVE_FTR_H","H_MEANS_FIVE_HC","H_MEANS_FIVE_HF","H_MEANS_FIVE_HR","H_MEANS_FIVE_HS","H_MEANS_FIVE_HST","H_MEANS_FIVE_HTAG","H_MEANS_FIVE_HTHG","H_MEANS_FIVE_HTR_A","H_MEANS_FIVE_HTR_D","H_MEANS_FIVE_HTR_H","H_MEANS_FIVE_HY","A_MEANS_THREE_AC","A_MEANS_THREE_AF","A_MEANS_THREE_AR","A_MEANS_THREE_AS","A_MEANS_THREE_AST","A_MEANS_THREE_AY","A_MEANS_THREE_FTAG","A_MEANS_THREE_FTHG","A_MEANS_THREE_FTR_A","A_MEANS_THREE_FTR_D","A_MEANS_THREE_FTR_H","A_MEANS_THREE_HC","A_MEANS_THREE_HF","A_MEANS_THREE_HR","A_MEANS_THREE_HS","A_MEANS_THREE_HST","A_MEANS_THREE_HTAG","A_MEANS_THREE_HTHG","A_MEANS_THREE_HTR_A","A_MEANS_THREE_HTR_D","A_MEANS_THREE_HTR_H","A_MEANS_THREE_HY","H_MEANS_THREE_AC","H_MEANS_THREE_AF","H_MEANS_THREE_AR","H_MEANS_THREE_AS","H_MEANS_THREE_AST","H_MEANS_THREE_AY","H_MEANS_THREE_FTAG","H_MEANS_THREE_FTHG","H_MEANS_THREE_FTR_A","H_MEANS_THREE_FTR_D","H_MEANS_THREE_FTR_H","H_MEANS_THREE_HC","H_MEANS_THREE_HF","H_MEANS_THREE_HR","H_MEANS_THREE_HS","H_MEANS_THREE_HST","H_MEANS_THREE_HTAG","H_MEANS_THREE_HTHG","H_MEANS_THREE_HTR_A","H_MEANS_THREE_HTR_D","H_MEANS_THREE_HTR_H","H_MEANS_THREE_HY","A_STD_FIVE_AC","A_STD_FIVE_AF","A_STD_FIVE_AR","A_STD_FIVE_AS","A_STD_FIVE_AST","A_STD_FIVE_AY","A_STD_FIVE_FTAG","A_STD_FIVE_FTHG","A_STD_FIVE_FTR_A","A_STD_FIVE_FTR_D","A_STD_FIVE_FTR_H","A_STD_FIVE_HC","A_STD_FIVE_HF","A_STD_FIVE_HR","A_STD_FIVE_HS","A_STD_FIVE_HST","A_STD_FIVE_HTAG","A_STD_FIVE_HTHG","A_STD_FIVE_HTR_A","A_STD_FIVE_HTR_D","A_STD_FIVE_HTR_H","A_STD_FIVE_HY","H_STD_FIVE_AC","H_STD_FIVE_AF","H_STD_FIVE_AR","H_STD_FIVE_AS","H_STD_FIVE_AST","H_STD_FIVE_AY","H_STD_FIVE_FTAG","H_STD_FIVE_FTHG","H_STD_FIVE_FTR_A","H_STD_FIVE_FTR_D","H_STD_FIVE_FTR_H","H_STD_FIVE_HC","H_STD_FIVE_HF","H_STD_FIVE_HR","H_STD_FIVE_HS","H_STD_FIVE_HST","H_STD_FIVE_HTAG","H_STD_FIVE_HTHG","H_STD_FIVE_HTR_A","H_STD_FIVE_HTR_D","H_STD_FIVE_HTR_H","H_STD_FIVE_HY","A_STD_THREE_AC","A_STD_THREE_AF","A_STD_THREE_AR","A_STD_THREE_AS","A_STD_THREE_AST","A_STD_THREE_AY","A_STD_THREE_FTAG","A_STD_THREE_FTHG","A_STD_THREE_FTR_A","A_STD_THREE_FTR_D","A_STD_THREE_FTR_H","A_STD_THREE_HC","A_STD_THREE_HF","A_STD_THREE_HR","A_STD_THREE_HS","A_STD_THREE_HST","A_STD_THREE_HTAG","A_STD_THREE_HTHG","A_STD_THREE_HTR_A","A_STD_THREE_HTR_D","A_STD_THREE_HTR_H","A_STD_THREE_HY","H_STD_THREE_AC","H_STD_THREE_AF","H_STD_THREE_AR","H_STD_THREE_AS","H_STD_THREE_AST","H_STD_THREE_AY","H_STD_THREE_FTAG","H_STD_THREE_FTHG","H_STD_THREE_FTR_A","H_STD_THREE_FTR_D","H_STD_THREE_FTR_H","H_STD_THREE_HC","H_STD_THREE_HF","H_STD_THREE_HR","H_STD_THREE_HS","H_STD_THREE_HST","H_STD_THREE_HTAG","H_STD_THREE_HTHG","H_STD_THREE_HTR_A","H_STD_THREE_HTR_D","H_STD_THREE_HTR_H","H_STD_THREE_HY"]
best_features_NB = ['A_MEANS_FIVE_AC', 'A_MEANS_FIVE_AS', 'A_MEANS_FIVE_AST','A_MEANS_FIVE_FTAG', 'A_MEANS_FIVE_FTHG', 'A_MEANS_FIVE_FTR_H','A_MEANS_FIVE_HC', 'A_MEANS_FIVE_HS', 'A_MEANS_FIVE_HST','A_MEANS_FIVE_HTR_A', 'H_MEANS_FIVE_AC', 'H_MEANS_FIVE_AS','H_MEANS_FIVE_AST', 'H_MEANS_FIVE_AY', 'H_MEANS_FIVE_FTAG','H_MEANS_FIVE_FTHG', 'H_MEANS_FIVE_FTR_A', 'H_MEANS_FIVE_FTR_H','H_MEANS_FIVE_HC', 'H_MEANS_FIVE_HS', 'H_MEANS_FIVE_HST','H_MEANS_FIVE_HTR_H', 'A_MEANS_THREE_AC', 'A_MEANS_THREE_AS','A_MEANS_THREE_FTHG', 'A_MEANS_THREE_HS', 'H_MEANS_THREE_AS','A_STD_FIVE_HF', 'H_STD_FIVE_HC', 'H_STD_FIVE_HST']
features_list = [
    ['best_features_MLP', best_features_MLP],
    ['all_features', all_features],
    ['best_features_NB', best_features_NB]
]

In [5]:
# Construct base layer
base_layer = [
    ['XGBoost', False, 'no', 9, XGBClassifier(
        learning_rate=0.01,
        n_estimators=200,
        max_depth=4,
        min_child_weight=1,
        gamma=0.41,
        subsample=0.76,
        colsample_bytree=0.4,
        objective='multi:softprob',
        reg_alpha=0.46,
        nthread=4,
        scale_pos_weight=1,
        seed=15), 
     ['all_features', all_features]
    ],
    ['NB', True, 'no', 9, GaussianNB(), ['best_features_NB', best_features_NB]],
    ['MLP', True, 'no', 9, MLPClassifier(
        random_state=0,
        activation='logistic', 
        alpha=1.4, 
        hidden_layer_sizes=(160,),
        max_iter=260, 
        solver='sgd'),
     ['best_features_MLP', best_features_MLP]
    ],
]

In [6]:
# Configure number of fold
NFOLDS = 4

## Import Data

In [7]:
# DB Sqlite connection
import sqlite3
db = "/Users/thibaultclement/Project/ligue1-predict/src/notebook/data/db/soccer_predict.sqlite"
conn = sqlite3.connect(db)
cur = conn.cursor()

In [8]:
# Get all prematch data
df_all = pd.read_sql_query("SELECT * FROM pre_matchs ORDER BY INFO_Date ASC;", conn)
df_all = (df_all[df_all.columns.drop(['index'])])
df_all.shape

(37907, 190)

In [9]:
# Remove all game between June (include) and October (include)
df_all['INFO_Date'] = pd.to_datetime(df_all['INFO_Date'])
df_all['INFO_Date'].dt.month
df_all = df_all[(df_all['INFO_Date'].dt.month < 6) | (df_all['INFO_Date'].dt.month >= 10)]
df_all.shape

(30912, 190)

In [10]:
# Create a INFO_WIN column containing the gain if you bet the good result
df_all['INFO_WIN'] = 0
df_all.loc[df_all.INFO_FTR == 'H', 'INFO_WIN'] = df_all[odd_H]
df_all.loc[df_all.INFO_FTR == 'A', 'INFO_WIN'] = df_all[odd_A]
df_all.loc[df_all.INFO_FTR == 'D', 'INFO_WIN'] = df_all[odd_D]
df_all['INFO_WIN_P'] = 0
df_all.loc[df_all.INFO_FTR == 'H', 'INFO_WIN_P'] = df_all['INFO_PSH']
df_all.loc[df_all.INFO_FTR == 'A', 'INFO_WIN_P'] = df_all['INFO_PSA']
df_all.loc[df_all.INFO_FTR == 'D', 'INFO_WIN_P'] = df_all['INFO_PSD']

## Methods

In [11]:
def get_dataset(league, season, calibration, historical_training_year, features):
    # Filter by league
    df = df_all[(df_all['INFO_Div'] == league)]
    # Keep season for test and filter by number of historical season used to train
    date_start_learn = datetime.date(season-historical_training_year, 8, 1)
    date_end_learn = datetime.date(season, 8, 1)
    date_start_test_season = datetime.date(season, 8, 1)
    date_end_test_season = datetime.date(season+1, 8, 1)
    df_test = df[(df['INFO_Date'] > date_start_test_season)]
    df_test = df_test[(df_test['INFO_Date'] < date_end_test_season)]
    df = df[(df['INFO_Date'] > date_start_learn)]
    df = df[(df['INFO_Date'] < date_end_learn)]
    # reset index
    df = df.reset_index()
    df_test = df_test.reset_index()
    # Encode Label
    le = preprocessing.LabelEncoder()
    le.fit(df[target])
    # Filter by feature used to train
    X = pd.get_dummies(df[features])
    #y = le.transform(df[target])
    y = df[target]
    X_test_season = pd.get_dummies(df_test[features])
    #y_test_season = le.transform(df_test[target])
    y_test_season = df_test[target]
    # Impute of missing values (NaN) with the mean
    # TODO drop NaN instead of replacing ith means 
    imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
    imp = imp.fit(X)
    X = imp.transform(X)
    X_test_season = imp.transform(X_test_season)
    # Standardize features
    if calibration:
        sc_X = StandardScaler().fit(X)
        X = sc_X.transform(X)
        X_test_season = sc_X.transform(X_test_season)
    return df, df_test, X, y, X_test_season, y_test_season, le

In [ ]:
def get_oof(clf, x_train, x_test, kf_param):
    oof_train = np.zeros((x_train.shape[0],3))
    oof_test = np.zeros((x_test.shape[0],3))
    oof_test_skf = np.empty((NFOLDS, x_test.shape[0], 3))

    for i, (train_index, test_index) in enumerate(kf_param):
        x_tr = x_train[train_index]
        y_tr = y_train_df[train_index]
        x_te = x_train[test_index]
        # Calibrate model
        clf.fit(x_tr, y_tr)

        oof_train[test_index] = clf.predict_proba(x_te)
        oof_test_skf[i, :] = clf.predict_proba(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train, oof_test

In [12]:
def get_layer_columns(layer, classes):
    cols = []
    for clf_name, preprocess, calibration, historical_training_year, clf, features in layer:
        for result in classes:
            cols.append(clf_name+result)
    return cols

In [13]:
def get_layer1_df(X, X_test_season, base_layer, cols):
    X_train_layer1 = np.zeros((X.shape[0], len(base_layer)*3))
    X_train_layer1 = pd.DataFrame(X_train_layer1, columns=cols)
    X_test_layer1 = np.zeros((X_test_season.shape[0], len(base_layer)*3))
    X_test_layer1 = pd.DataFrame(X_test_layer1, columns=cols)
    return X_train_layer1, X_test_layer1

In [14]:
def get_score(y, pred, probs):
    # Compute cross-entropy score
    ll = log_loss(y, probs)
    # Compute accuracy score
    acc = accuracy_score(y, pred)
    # Compute precision score
    prec = precision_score(y, pred, average=None)
    prec_A = prec[0]
    prec_D = prec[1]
    prec_H = prec[2]
    # Compute recall score
    rec = recall_score(y, pred, average=None)
    rec_A = rec[0]
    rec_D = rec[1]
    rec_H = rec[2]
    # Compute F1 score
    f1 = f1_score(y, pred, average=None)
    f1_A = f1[0]
    f1_D = f1[1]
    f1_H = f1[2]
    return {
        'll': ll, 
        'acc': acc, 
        'prec_A': prec_A, 
        'prec_D': prec_D, 
        'prec_H': prec_H, 
        'rec_A': rec_A, 
        'rec_D': rec_D, 
        'rec_H': rec_H, 
        'f1_A': f1_A, 
        'f1_D': f1_D, 
        'f1_H': f1_H
    }

In [15]:
def get_money(df_test, pred_season, prob_season):
    # Join odd and prediction together
    df_test_season = df_test
    df_test_season['probs_A'] = prob_season[:,0]
    df_test_season['probs_D'] = prob_season[:,1]
    df_test_season['probs_H'] = prob_season[:,2]
    df_test_season['probs'] = df_test_season[['probs_A','probs_D','probs_H']].max(axis=1)
    #df_test_season['pred'] = le.inverse_transform(pred_season)
    df_test_season['pred'] = pred_season
    df_test_season['WIN'] = -1
    df_test_season.loc[df_test_season.INFO_FTR == df_test_season.pred, 'WIN'] = df_test_season['INFO_WIN']-1
    df_test_season['WIN_P'] = -1
    df_test_season.loc[df_test_season.INFO_FTR == df_test_season.pred, 'WIN_P'] = df_test_season['INFO_WIN_P']-1
    df_test_season['INFO_ODD'] = 0
    df_test_season.loc[df_test_season.pred == 'A', 'INFO_ODD_BET'] = df_test_season[odd_A]
    df_test_season.loc[df_test_season.pred == 'D', 'INFO_ODD_BET'] = df_test_season[odd_D]
    df_test_season.loc[df_test_season.pred == 'H', 'INFO_ODD_BET'] = df_test_season[odd_H]
    df_test_season['prob_less_bet'] = 0
    df_test_season.loc[df_test_season.pred == 'A', 'prob_less_bet'] = df_test_season['probs'] - df_test_season[odd_A].apply(lambda x: 1/x)
    df_test_season.loc[df_test_season.pred == 'D', 'prob_less_bet'] = df_test_season['probs'] - df_test_season[odd_D].apply(lambda x: 1/x)
    df_test_season.loc[df_test_season.pred == 'H', 'prob_less_bet'] = df_test_season['probs'] - df_test_season[odd_H].apply(lambda x: 1/x)
    # calculate money I can get following different scenario
    # Bet on all
    bet_all = df_test_season.WIN.mean()
    # Bet under 1.9
    bet_lte_19 = df_test_season[df_test_season['INFO_ODD_BET'] < 1.9].WIN.mean()
    # Bet under 4
    bet_lte_4 = df_test_season[df_test_season['INFO_ODD_BET'] < 4].WIN.mean()
    # Bet between 1.9 and 4
    bet_btw_19_4 = df_test_season[(df_test_season['INFO_ODD_BET'] > 1.9) & (df_test_season['INFO_ODD_BET'] < 4)].WIN.mean()
    # Bet between 1.9 and 5
    bet_btw_19_5 = df_test_season[(df_test_season['INFO_ODD_BET'] > 1.9) & (df_test_season['INFO_ODD_BET'] < 5)].WIN.mean()
    # Bet between 1.5 and 4
    bet_btw_15_4 = df_test_season[(df_test_season['INFO_ODD_BET'] > 1.5) & (df_test_season['INFO_ODD_BET'] < 4)].WIN.mean()
    # Bet between 1.5 and 5
    bet_btw_15_5 = df_test_season[(df_test_season['INFO_ODD_BET'] > 1.5) & (df_test_season['INFO_ODD_BET'] < 5)].WIN.mean()
    # Bet prob higher than 50%
    bet_pred_gte_50 = df_test_season[df_test_season.probs > 0.5].WIN.mean()
    # Bet prob higher than 60%
    bet_pred_gte_60 = df_test_season[df_test_season.probs > 0.6].WIN.mean()
    # Bet prob higher than 70%
    bet_pred_gte_70 = df_test_season[df_test_season.probs > 0.7].WIN.mean()
    return {
        'bet_all': bet_all,
        'bet_lte_19': bet_lte_19,
        'bet_lte_4': bet_lte_4,
        'bet_btw_19_4': bet_btw_19_4,
        'bet_btw_19_5': bet_btw_19_5,
        'bet_btw_15_4': bet_btw_15_4,
        'bet_btw_15_5': bet_btw_15_5,
    }


## Loop on season

In [16]:
# Init dataframe
result_df = pd.DataFrame(columns=[
    'league', 
    'season', 
    'historical_training_year', 
    'C',
    'penalty',
    'class_weight',
    'solver',
    'max_iter',
    'll',
    'acc',
    'prec_A',
    'prec_D',
    'prec_H',
    'rec_A',
    'rec_D',
    'rec_H',
    'f1_A',
    'f1_D',
    'f1_H',
    'bet_all',
    'bet_lte_19',
    'bet_lte_4',
    'bet_btw_19_4',
    'bet_btw_19_5',
    'bet_btw_15_4',
    'bet_btw_15_5'
])

In [17]:
for season in season_list:
    
    # Prepare the layer 1
    df, df_test, X, y, X_test_season, y_test_season, le = get_dataset(league, season, False, 9, all_features)
    cols = get_layer_columns(base_layer, classes)
    X_train_layer1, X_test_season_layer1 = get_layer1_df(X, X_test_season, base_layer, cols)
    
    # Process base layer
    for clf_name, preprocess, calibration, historical_training_year, classifier, features in base_layer:
        print league,str(season),clf_name,str(historical_training_year)
        
        #Get the dataset
        df, df_test, X, y, X_test_season, y_test_season, le = get_dataset(league, season, calibration, historical_training_year, features[1])
        
        # Check if we need to recalibrate the prediction
        if calibration == 'sigmoid':
            clf = CalibratedClassifierCV(classifier, cv=4, method='sigmoid')
        elif calibration == 'isotonic':
            clf = CalibratedClassifierCV(classifier, cv=4, method='isotonic')
        elif calibration == 'no':
            clf = classifier
        
        # Calibrate the model
        clf.fit(X, y)
        
        # Predict probabilities of the test season
        y_probs_train_layer1 = clf.predict_proba(X)
        y_probs_test_season_layer1 = clf.predict_proba(X_test_season)
        
        # Add this model prediction to layer 1 dataset
        X_train_layer1.loc[:, [clf_name+result for result in classes]] = y_probs_train_layer1
        X_test_season_layer1.loc[:, [clf_name+result for result in classes]] = y_probs_test_season_layer1
    
    #Process Layer 1
    clf_1 = LogisticRegression(
        C=best_params['C'],
        penalty=best_params['penalty'],
        class_weight=best_params['class_weight'],
        solver=best_params['solver'],
        max_iter=best_params['max_iter'],
        multi_class=best_params['multi_class'],
    )
    clf_1.fit(X_train_layer1, y)
    # Predict target values
    y_pred = clf_1.predict(X_test_season_layer1)
    # Predict probabilities
    y_probs = clf_1.predict_proba(X_test_season_layer1)
    # get scores
    score_dict = get_score(y_test_season, y_pred, y_probs)
    # get money earned
    money_dict = get_money(df_test, y_pred, y_probs)
    # Add all info to result dataframe
    result_df.loc[len(result_df.index)] = [
        league, 
        season, 
        historical_training_year, 
        best_params['C'],
        best_params['penalty'],
        best_params['class_weight'],
        best_params['solver'],
        best_params['max_iter'],
        score_dict['ll'],
        score_dict['acc'],
        score_dict['prec_A'],
        score_dict['prec_D'],
        score_dict['prec_H'],
        score_dict['rec_A'],
        score_dict['rec_D'],
        score_dict['rec_H'],
        score_dict['f1_A'],
        score_dict['f1_D'],
        score_dict['f1_H'],
        money_dict['bet_all'],
        money_dict['bet_lte_19'],
        money_dict['bet_lte_4'],
        money_dict['bet_btw_19_4'],
        money_dict['bet_btw_19_5'],
        money_dict['bet_btw_15_4'],
        money_dict['bet_btw_15_5']
    ]
    

D1 2014 XGBoost 9
D1 2014 NB 9
D1 2014 MLP 9
D1 2015 XGBoost 9
D1 2015 NB 9
D1 2015 MLP 9
D1 2016 XGBoost 9
D1 2016 NB 9
D1 2016 MLP 9


# Save result

In [18]:
result_df.to_csv('./report/STACKING_1_FTR_D1_BEST_HYPERPARAM_LOG_LOSS-VALIDATION.csv')

### Final result
Best is with ??? and ??? years of history

In [21]:
result_df

,league,season,historical_training_year,C,penalty,class_weight,solver,max_iter,ll,acc,...,f1_A,f1_D,f1_H,bet_all,bet_lte_19,bet_lte_4,bet_btw_19_4,bet_btw_19_5,bet_btw_15_4,bet_btw_15_5
0,D1,2014,9,8.291,l2,None,sag,270,1.342474,0.456349,...,0.419580,0.206186,0.568182,-0.005833,-0.060244,-0.008966,0.025906,0.031739,0.015351,0.020761
1,D1,2015,9,8.291,l2,None,sag,270,1.455315,0.473251,...,0.432432,0.288660,0.572614,0.016914,-0.015060,0.002203,0.012153,0.024645,-0.005580,0.005521
2,D1,2016,9,8.291,l2,None,sag,270,1.542562,0.350000,...,0.316456,0.192982,0.443548,-0.202308,-0.326528,-0.113829,-0.011733,-0.090552,-0.095319,-0.153831


In [20]:
X_train_layer1

,XGBoostA,XGBoostD,XGBoostH,NBA,NBD,NBH,MLPA,MLPD,MLPH
0,0.279969,0.243720,0.476311,0.144821,0.260931,0.594248,0.306964,0.256063,0.436973
1,0.432669,0.248423,0.318907,0.938504,0.055820,0.005676,0.445320,0.238716,0.315964
2,0.439778,0.265275,0.294947,0.956420,0.042407,0.001173,0.460049,0.260201,0.279750
3,0.234580,0.327127,0.438293,0.080766,0.268106,0.651128,0.181941,0.226449,0.591610
4,0.255676,0.241452,0.502872,0.000013,0.000494,0.999493,0.143772,0.176581,0.679647
5,0.275133,0.240479,0.484389,0.003947,0.029953,0.966100,0.236894,0.221570,0.541536
6,0.284215,0.287574,0.428211,0.300224,0.589584,0.110192,0.371066,0.284059,0.344875
7,0.164030,0.193751,0.642219,0.000047,0.000584,0.999369,0.171993,0.183573,0.644434
8,0.315732,0.272952,0.411315,0.418144,0.322031,0.259825,0.332488,0.247499,0.420013
9,0.328320,0.278061,0.393619,0.371676,0.518101,0.110223,0.358275,0.274912,0.366813
